# Artistic Image Generation Using Generative Adverserial Networks


***

## Next --- shells are used only if you are using Google Colab platform to connect your google drive

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
import os
os.chdir('drive/seminar') #locate your folder in drive
os.getcwd()

## Alternative way to connect to your google drive

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/My Drive/seminar') #locate your folder in drive this is main
                                            #difference in the method to mount the drive
os.getcwd()


***

## Install Dependacies required for Program on Google colab

In [0]:

!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

import tensorflow as tf
print(tf.test.gpu_device_name())
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['CUDA_VISIBLE_DEVICES']


/device:GPU:0


'0'

In [0]:
!ls 

ARTGAN.ipynb  LandGAN.ipynb  newLand2  __pycache__	   train_64
data	      model	     newLand3  seminar report.odt  train_landscape_128
drive	      model2	     newLand4  tensorboard	   utils.py
GANART.ipynb  newLand	     newLand5  train_128


*** 
# IF your are running in your on computer the follow from here

 ***
### *Note* - please chech the current folder using os library and folder name. If folder is not present in your computer create on in current directory
### For Tensorboard uncomment the code related tensorboard
***

In [ ]:

import os
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
import scipy.misc
import datetime

HEIGHT, WIDTH, CHANNEL = 128, 128, 3
BATCH_SIZE = 64
EPOCH = 5000
os.environ['CUDA_VISIBLE_DEVICES']='0'
version = "newLand4" #-----newLand4
newLand_path = version

def lrelu(x, n, leak=0.2):
  return tf.maximum(x, leak * x, name=n)


def process_data():
    current_dir = os.getcwd()
    landscape_dir = os.path.join(current_dir, 'train_landscape_128')
    images = []
    for each in os.listdir(landscape_dir):
        images.append(os.path.join(landscape_dir, each))

    all_images = tf.convert_to_tensor(images, dtype=tf.string)

    images_queue = tf.train.slice_input_producer([all_images])
    
    content = tf.read_file(images_queue[0])
    image = tf.image.decode_jpeg(content, channels=CHANNEL)

    #--temp-----------
    image = tf.image.random_flip_left_right(image)
    
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    
    size = [HEIGHT, WIDTH]
    
    image = tf.image.resize_images(image, size)
    image.set_shape([HEIGHT, WIDTH, CHANNEL])
    

    image = tf.cast(image, tf.float32)
    image = image/255.0
    

    iamges_batch = tf.train.shuffle_batch([image],
                                         batch_size=BATCH_SIZE,
                                         num_threads = 4,
                                         capacity = 2000 + 3*BATCH_SIZE,
                                         min_after_dequeue = 2000)
    
    num_images = len(images)
    return iamges_batch, num_images
    


def generator(input, random_dim, is_train, reuse=False):

    c4, c8, c16, c32, c64 = 512, 256, 128, 64, 32
    s4 = 4
    
    output_dim = CHANNEL
    with tf.variable_scope('gen') as scope:
        if reuse:
            scope.reuse_variables()

        w1 = tf.get_variable('w1', shape=[random_dim, s4*s4*c4],dtype=tf.float32,initializer = tf.truncated_normal_initializer(stddev=0.02))
        b1 = tf.get_variable('b1', shape=[c4*s4*s4], dtype=tf.float32,initializer = tf.constant_initializer(0.0))
        flat_conv1 = tf.add( tf.matmul(input, w1), b1,name="flat_conv1")

    
        #4x4x512
        conv1 = tf.reshape(flat_conv1, shape=[-1, s4, s4, c4], name='conv1')
        bn1 = tf.contrib.layers.batch_norm(conv1, is_training=is_train,epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn1')
        act1 = tf.nn.relu(bn1, name='act1')
        #------------
        #tf.summary.histogram("g_act1", act1)
        #------------

        #8x8x256
        conv2 = tf.layers.conv2d_transpose(act1, c8, kernel_size=[5,5], strides=[2,2], padding="SAME",
                                          kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn2')
        act2 = tf.nn.relu(bn2, name='act2')
        #------------
        #tf.summary.histogram("g_act2", act2)
        #------------
        

        #16x16x128
        conv3 = tf.layers.conv2d_transpose(act2, c16, kernel_size=[5,5], strides=[2,2], padding="SAME",
                                          kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn3')
        act3 = tf.nn.relu(bn3, name='act3')
        #------------
        #tf.summary.histogram("g_act3", act3)
        #------------
        
        #32x32x64
        conv4 = tf.layers.conv2d_transpose(act3, c32, kernel_size=[5,5], strides=[2,2], padding="SAME",
                                          kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn4')
        act4 = tf.nn.relu(bn4, name='act4')
        #------------
        #tf.summary.histogram("g_act4", act4)
        #------------

        #64x64x32
        conv5 = tf.layers.conv2d_transpose(act4, c64, kernel_size=[5,5], strides=[2,2], padding="SAME",
                                          kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), name='conv5')
        bn5 = tf.contrib.layers.batch_norm(conv5, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn5')
        act5 = tf.nn.relu(bn5, name='act5')
        #------------
        #tf.summary.histogram("g_act5", act5)
        #------------
        

        #128x128x3
        conv6 = tf.layers.conv2d_transpose(act5, output_dim, kernel_size=[5,5], strides=[2,2], padding="SAME",
                                          kernel_initializer = tf.truncated_normal_initializer(stddev=0.02), name='conv6')
        #bn6 = tf.contrib.layers.batch_norm(conv6, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, name='bn6')
        act6 = tf.nn.tanh(conv6, name='act6')
        #------------
        #tf.summary.histogram("g_act6", act6)
        #------------
        
        return act6

    
def discriminator(input, is_train, reuse=False):
    c2, c4, c8, c16 = 64, 128, 256, 512
    with tf.variable_scope('dis') as scope:

        if reuse:
            scope.reuse_variables()

        #64x64x64
        conv1 = tf.layers.conv2d(input, c2, kernel_size=[5,5], strides=[2, 2], padding="SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02), name='conv1')
        #bn1 = tf.contrib.layers.batch_norm(conv1, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn1')
        act1 = lrelu(conv1, n='act1')
        #------------
        #tf.summary.histogram("d_act1", act1)
        #------------

        #32x32x128
        conv2 = tf.layers.conv2d(act1, c4, kernel_size=[5,5], strides=[2, 2], padding="SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02), name='conv2')
        bn2 = tf.contrib.layers.batch_norm(conv2, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn2')
        act2 = lrelu(bn2, n='act2')
        #------------
        #tf.summary.histogram("d_act2", act2)
        #------------

        #16x16x256
        conv3 = tf.layers.conv2d(act2, c8, kernel_size=[5,5], strides=[2, 2], padding="SAME",
                                kernel_initializer=tf.truncated_normal_initializer(stddev=0.02), name='conv3')
        bn3 = tf.contrib.layers.batch_norm(conv3, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn3')
        act3 = lrelu(bn3, n='act3')
        #------------
        #tf.summary.histogram("d_act3", act3)
        #------------

        #8x8x512
        conv4 = tf.layers.conv2d(act3, c16, kernel_size=[5,5], strides=[2, 2], padding="SAME",
                               kernel_initializer=tf.truncated_normal_initializer(stddev=0.02), name='conv4')
        bn4 = tf.contrib.layers.batch_norm(conv4, is_training=is_train, epsilon=1e-5, decay=0.9, updates_collections=None, scope='bn4')
        act4 = lrelu(bn4, n='act4')
        #------------
        #tf.summary.histogram("d_act4", act4)
        #------------

        dim = int(np.prod(act4.get_shape()[1:]))
        fc1 = tf.reshape(act4, shape=[-1, dim], name='fc1')

        w2 = tf.get_variable('w2', shape=[fc1.shape[-1], 1], dtype=tf.float32,
                        initializer = tf.truncated_normal_initializer(stddev=0.02))

        b2 = tf.get_variable('b2', shape=[1], dtype=tf.float32, 
                        initializer = tf.constant_initializer(0.0))
        logits = tf.add(tf.matmul(fc1, w2), b2, name='logits')
        acted_out = tf.nn.sigmoid(logits)
        return logits, acted_out

#----plot single-----------
def plot_single(samples):
  fig = plt.figure(figsize=(8,8))
  gs = gridspec.GridSpec(2,2)
  gs.update(hspace=0.05, wspace=0.05)

  for i, sample in enumerate(samples):
    ax = plt.subplot(gs[i])
    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_aspect('equal')
    
    plt.imshow(((sample/2)+0.5), cmap='Greys_r')
    
    
#-----------------------------


def plot(samples):
  fig = plt.figure(figsize=(8,8))
  gs = gridspec.GridSpec(8,8)
  gs.update(hspace=0.05, wspace=0.05)

  for i, sample in enumerate(samples):
    ax = plt.subplot(gs[i])
    plt.axis('off')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_aspect('equal')
    
    plt.imshow(((sample/2)+0.5), cmap='Greys_r')
    
    
  return fig


  
def train():
    random_dim = 100
    print((os.environ["CUDA_VISIBLE_DEVICES"]))

  
    with tf.variable_scope('input'):
        real_image = tf.placeholder(tf.float32, shape=[None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='random_input')
        is_train = tf.placeholder(tf.bool, name='is_train')

    
    fake_image = generator(random_input, random_dim, is_train)
    sample_fake = generator(random_input, random_dim, is_train, reuse=True)
    real_logits, real_result = discriminator(real_image, is_train)
    fake_logits, fake_result = discriminator(fake_image, is_train, reuse=True)
  
    d_loss1 = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits( logits=real_logits, labels = tf.ones_like(real_logits)))
    d_loss2 = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits( logits=fake_logits, labels = tf.zeros_like(fake_logits)))
    d_loss = d_loss1 + d_loss2
  
    g_loss = tf.reduce_mean( tf.nn.sigmoid_cross_entropy_with_logits( logits=fake_logits, labels = tf.ones_like(fake_logits)))
  
    #------------------
    #tf.summary.scalar("d_loss1",d_loss1)
    #tf.summary.scalar("d_loss2",d_loss2)
    #tf.summary.scalar("d_loss", d_loss)
    #tf.summary.scalar("g_loss", g_loss)
    #------------------
    t_vars = tf.trainable_variables()
    d_vars = [ var for var in t_vars if 'dis' in var.name ]
    g_vars = [ var for var in t_vars if 'gen' in var.name ]
    
  
    trainer_d = tf.train.RMSPropOptimizer(learning_rate = 2e-4).minimize(d_loss, var_list=d_vars)
    trainer_g = tf.train.RMSPropOptimizer(learning_rate = 2e-4).minimize(g_loss, var_list=g_vars)
  

    d_clip = [v.assign(tf.clip_by_value(v, -0.01, 0.01)) for v in d_vars]
  
    batch_size = BATCH_SIZE
    image_batch, samples_num = process_data()
  
    batch_num = int(samples_num/batch_size)
    total_batch = 0 
    sess = tf.Session()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
  
    ckpt = tf.train.latest_checkpoint('model2/'+version)
    if ckpt!= None:
      saver.restore(sess, ckpt)
      
    #----------------
    #train_writer = tf.summary.FileWriter("tensorboard/1/train", sess.graph)
    #----------------
        

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
  
    print('total training samples num : %d'% samples_num)
    print('batch_size : %d , num Batch : %d, epochs : %d'%(batch_size, batch_num, EPOCH))
    print('start training ...............')
    

    for i in range(EPOCH):
    #for i in range(EPOCH):
      print(i)
      print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
      #s=""
      for j in range(batch_num):
      
        #-------------
        #merge = tf.summary.merge_all()
        #-------------
        
        train_noise = np.random.uniform(-1.0, 1.0, size=[batch_size, random_dim]).astype(np.float32)
        
        train_image = sess.run(image_batch)
     
        #-----------------clip------
        sess.run(d_clip)
      
        _, dLoss = sess.run([trainer_d, d_loss], feed_dict={random_input:train_noise, real_image:train_image, is_train:True})
        _, gLoss = sess.run([trainer_g, g_loss], feed_dict={random_input:train_noise, is_train:True})
        #summary = sess.run(merge, feed_dict={random_input:train_noise, real_image:train_image, is_train:True})
        #----------------
        #summary,_, dLoss,_, gLoss = sess.run([merge,trainer_d, d_loss, trainer_g, g_loss], feed_dict={random_input:train_noise, real_image:train_image, is_train:True})
        #------------------------------
        #s = summary
        #if j==233:
        ## train_writer.add_summary(summary, i)
        
      #train_writer.add_summary(s, i)

      if i== 0 or i%30==0 :
        print("saving model..........")
        if not os.path.exists('model2/'+version):
          os.makedirs('model2/'+version)
        saver.save(sess,'model2/'+version+'/'+str(i))
 
      if i%30 == 0:
        if not os.path.exists(newLand_path):
          os.makedirs(newLand_path)
          
        
      sample_noise = np.random.uniform(-1.0, 1.0, size=[64, random_dim]).astype(np.float32)
      imgtest = sess.run(fake_image, feed_dict={random_input:sample_noise, is_train:False})
      
      print('Train:[%d], d_loss:%f, g_loss:%f'%(i, dLoss, gLoss))

      
      fig = plot(imgtest)
      plt.savefig('newLand4/land_{}.jpg'.format(str(i)+"_"+"d_"+str(dLoss)+"_g_"+str(gLoss)), bbox_inches='tight')
      plt.close()
      
      #--------plot single-------------
      sample_noise_single = np.random.uniform(-1.0, 1.0, size=[4, random_dim]).astype(np.float32)
      imgtest_single = sess.run(fake_image, feed_dict={random_input:sample_noise_single, is_train:False})

      
      fig = plot_single(imgtest_single)
      plt.savefig('newLand4/land2_{}.jpg'.format(str(i)+"_"+"d_"+str(dLoss)+"_g_"+str(gLoss)), bbox_inches='tight')
      plt.close()
      #----------------------------------


    coord.request_stop()
    coord.join(threads)
# def test():
    # random_dim = 100
    # with tf.variable_scope('input'):
        # real_image = tf.placeholder(tf.float32, shape = [None, HEIGHT, WIDTH, CHANNEL], name='real_image')
        # random_input = tf.placeholder(tf.float32, shape=[None, random_dim], name='rand_input')
        # is_train = tf.placeholder(tf.bool, name='is_train')
    
    # # wgan
    # fake_image = generator(random_input, random_dim, is_train)
    # real_result = discriminator(real_image, is_train)
    # fake_result = discriminator(fake_image, is_train, reuse=True)
    # sess = tf.InteractiveSession()
    # sess.run(tf.global_variables_initializer())
    # variables_to_restore = slim.get_variables_to_restore(include=['gen'])
    # print(variables_to_restore)
    # saver = tf.train.Saver(variables_to_restore)
    # ckpt = tf.train.latest_checkpoint('./model/' + version)
    # saver.restore(sess, ckpt)
    
if __name__ == "__main__":
      train()